In [1]:
import os
import pandas as pd
from tqdm import tqdm
import shutil
from src import utils
from scipy.stats import spearmanr
import math

In [2]:
# Project path
project_dir = os.path.abspath('')[:-9]
# Load enviromental variables

env_var = utils.load_env_variables(project_dir)
env_var["root_path"] = "/home/tpinho/IJGIS/Datasets/NSW_Election_2019"

In [3]:
dataset = "Original"
scv_methods = ["CrossValidation", 
               "Optimistic", 
             #  "RBuffer",
               "RegGBSCV_R_Kappa_0.0",
             #  "RegGBSCV_R_Kappa_0.1",
             #  "RegGBSCV_R_Kappa_0.2",
             #  "RegGBSCV_R_Kappa_0.3",
             #  "RegGBSCV_R_Kappa_0.4",
             #  "RegGBSCV_R_Kappa_0.5",
             #  "RegGBSCV_R_Kappa_0.6",
             #  "RegGBSCV_R_Kappa_0.7",
             #  "RegGBSCV_R_Kappa_0.8",
             #  "RegGBSCV_R_Kappa_0.9",
             #  "RegGBSCV_R_Kappa_1.0",
             #  "RegGBSCV_R_Kappa_3.0",
               "TraditionalSCV"] 
fs_methods = ["CFS", "LatLon"]

In [4]:
for scv in scv_methods:
    path = os.path.join(env_var["root_path"], dataset, "results", scv)
    ml_methods = ["KNN", "OLS", "Lasso", "Ridge", "ElasticNet", "DT", "LGBM", "RF", "MLP", "SVM"]
    for fs_method in tqdm(fs_methods):
        for ml_method in ml_methods:
            spearman = []
            metrics = pd.read_csv(os.path.join(path, "evaluations", fs_method, ml_method, "metrics.csv"))
            metrics.fillna(0, inplace=True)
            #metrics.fillna(0, inplace=True)
            metrics["SP_INV"] = 1 - metrics["SP"]
            metrics["MCPM"] = [0] * len(metrics)
            metrics.set_index("FOLD", inplace=True)
            folds = [file for file in os.listdir(os.path.join(path, "predictions", fs_method, ml_method))]
            #for fold in folds:
            #    predictions = pd.read_csv(os.path.join(path, "predictions", fs_method, ml_method, fold))
            #    print(predictions)
            #    rho, _ = spearmanr(predictions["GROUND_TRUTH"], predictions["PREDICTIONS"])
            #    metrics.loc[int(fold.split(".")[0]), "SP"] = rho
                #mcpm = metrics["RMSE"] * metrics["SPEARMAN"] * math.sin(2*math.pi/3)/2 
            metrics["MCPM"] = (metrics["MSE"]) * (metrics["SP_INV"]) * math.sin(2*math.pi/3)/2 + (metrics["MSE"]) * (metrics["STDE"]) * math.sin(2*math.pi/3)/2 + (metrics["STDE"]) * (metrics["SP_INV"]) * math.sin(2*math.pi/3)/2
            
            metrics.to_csv(os.path.join(path, "evaluations", fs_method, ml_method, "metrics_with_mcpm.csv"))
            
    metrics

100%|██████████| 2/2 [00:00<00:00,  7.38it/s]
